In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle
from StringIO import StringIO

import matplotlib
matplotlib.rcParams[u'font.family'] = [u'sans-serif']
matplotlib.rcParams[u'mathtext.default'] = u'regular'
matplotlib.rcParams[u'mathtext.it'] = u'sans\\-serif:oblique'
matplotlib.rcParams[u'mathtext.rm'] = u'serif'

import matplotlib.pyplot as plt
#import seaborn as sns
import seaborn_MLG as sns

pd.set_option('display.max_rows',1000)

%reload_ext autoreload
%matplotlib inline

/Volumes/Files/miniconda/envs/scienv2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
data1 = pd.read_csv('Data_peaks.csv',sep='\t').query('expt=="HE"')#.query('temp==283')
data2 = pd.read_csv('RNaseH_ZQ_DQ_Intensities_manuscript.csv',sep='\t').query('expt=="HE"')

In [10]:
def exponential_decay(par, xdata, ydata=None, yerr=None):
    
    # Parse multiple input parameter
    # formats for intensity, rate    
    if hasattr(par,'valuesdict'):
        # lmfit parameter format
        var = par.valuesdict()
        inten = var['inten']
        rate = var['rate']
    elif hasattr(par,'keys'):
        # dict format
        inten = par['inten']
        rate = par['rate']
    else:
        # array/list/tuple format
        inten = par[0]
        rate = par[1]

    # Calculate the y-data from the parameters
    ycalc = inten * np.exp(-1*rate*xdata)
    
    if ydata is None:
        # Calculation only
        return ycalc
    elif yerr is None:
        # Error minimization
        return ((ycalc - ydata)**2).sum()
    else:
        # Error minimization with weights
        return ((ycalc - ydata)**2/yerr**2).sum()

In [11]:
func = exponential_decay

# data = data1
# groupcols = ['name','part','temp']

data = data2
groupcols = ['name','part']

xname = 'time'
yname = 'height'

method = 'leastsq'
sigma = 0.95
threads = 4

params = [{'name':'inten', 'value':np.asarray(data.groupby(groupcols)[yname].max()), 'vary':True},
          {'name':'rate', 'value':20.0, 'vary':True}]

In [12]:
%autoreload 2
import pd_lmfit as analysis

zqdq = analysis.pdlmfit(data, func, groupcols, 
                        params, xname, yname, method='leastsq')

zqdq.fit()
zqdq.predict()

# Normalize intensities
zqdq.data['ydata'] = zqdq.data.ydata / zqdq.results.inten
zqdq.model['ycalc'] = zqdq.model.ycalc / zqdq.results.inten

try loop


In [66]:
## Attempt at calculating a jacobian

def dfunc(par, xs, ys):
    
    # Parse multiple input parameter
    # formats for intensity, rate    
    if hasattr(par,'valuesdict'):
        # lmfit parameter format
        var = par.valuesdict()
        inten = var['inten']
        rate = var['rate']
    elif hasattr(par,'keys'):
        # dict format
        inten = par['inten']
        rate = par['rate']
    else:
        # array/list/tuple format
        inten = par[0]
        rate = par[1]
        
        
    v = np.exp(-rate*xs)
    return np.array([v, -inten*xs*v, np.ones(len(xs))])
    #return -inten*xs*v

In [68]:
from scipy.optimize import minimize, fmin, check_grad

bdata = zqdq._fit_data.iloc[14]
xdata_ = np.asarray(bdata.xdata)
ydata_ = np.asarray(bdata.ydata)
params_ = bdata.params
minimizer_ = bdata.minimizer
bresult_ = zqdq.results.iloc[0]


p0 = np.array([ydata_.max(), 20.0])

minimize(exponential_decay, p0, args=(xdata_, ydata_), method='Nelder-Mead')#, options={"full_output":1})
#fmin(exponential_decay, p0, args=(xdata_, ydata_), full_output=True)

#minimize(exponential_decay, p0, args=(xdata_, ydata_), method='Newton-CG', jac=dfunc)#, options={"full_output":1})

  status: 0
    nfev: 239
 success: True
     fun: 676238076521.01135
       x: array([  2.95362888e+07,   2.74786355e+01])
 message: 'Optimization terminated successfully.'
     nit: 108